<a href="https://colab.research.google.com/github/ndhull24/College-Codes/blob/main/Parking_scheduling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from datetime import datetime, time

# Define weekly shifts (Mon–Fri)
WEEKDAYS = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
SHIFTS = [("09:00", "13:00"), ("13:00", "17:00"), ("17:00", "21:00")]

# Sample student input
students = [
    {
        "name": "Alice",
        "role": "Enforcement Officer",
        "class_schedule": {
            "Monday": [("10:00", "11:30")],
            "Tuesday": [],
            "Wednesday": [("13:30", "15:00")],
            "Thursday": [],
            "Friday": []
        }
    },
    {
        "name": "Bob",
        "role": "Shift Lead",
        "class_schedule": {
            "Monday": [],
            "Tuesday": [("09:00", "11:00")],
            "Wednesday": [],
            "Thursday": [("14:00", "16:00")],
            "Friday": []
        }
    },
    # Add more students...
]


In [2]:
def time_overlap(start1, end1, start2, end2):
    """Check if two time intervals overlap."""
    return not (end1 <= start2 or end2 <= start1)

def is_available(class_times, shift_start, shift_end):
    shift_start = datetime.strptime(shift_start, "%H:%M").time()
    shift_end = datetime.strptime(shift_end, "%H:%M").time()

    for cls_start, cls_end in class_times:
        cls_start = datetime.strptime(cls_start, "%H:%M").time()
        cls_end = datetime.strptime(cls_end, "%H:%M").time()
        if time_overlap(shift_start, shift_end, cls_start, cls_end):
            return False
    return True


In [3]:
from collections import defaultdict
import random

def assign_shifts(students):
    schedule = defaultdict(lambda: defaultdict(list))
    shift_counts = defaultdict(int)  # Keep track of shifts assigned to each student

    for day in WEEKDAYS:
        for shift_start, shift_end in SHIFTS:
            available_students = [
                s for s in students
                if is_available(s["class_schedule"].get(day, []), shift_start, shift_end)
                and shift_counts[s["name"]] < 5
            ]
            random.shuffle(available_students)

            # Assign based on role
            shift_roles = {"Shift Lead": None, "Office Lead": None, "Officers": []}
            for student in available_students:
                name = student["name"]
                role = student["role"]

                if role == "Shift Lead" and not shift_roles["Shift Lead"]:
                    shift_roles["Shift Lead"] = name
                    shift_counts[name] += 1
                elif role == "Office Lead" and not shift_roles["Office Lead"]:
                    shift_roles["Office Lead"] = name
                    shift_counts[name] += 1
                elif role == "Enforcement Officer" and len(shift_roles["Officers"]) < 3:
                    shift_roles["Officers"].append(name)
                    shift_counts[name] += 1

            schedule[day][f"{shift_start}-{shift_end}"] = shift_roles

    return dict(schedule)


In [4]:
import pprint
weekly_schedule = assign_shifts(students)
pprint.pprint(weekly_schedule)


{'Friday': defaultdict(<class 'list'>,
                       {'09:00-13:00': {'Office Lead': None,
                                        'Officers': [],
                                        'Shift Lead': None},
                        '13:00-17:00': {'Office Lead': None,
                                        'Officers': [],
                                        'Shift Lead': None},
                        '17:00-21:00': {'Office Lead': None,
                                        'Officers': [],
                                        'Shift Lead': None}}),
 'Monday': defaultdict(<class 'list'>,
                       {'09:00-13:00': {'Office Lead': None,
                                        'Officers': [],
                                        'Shift Lead': 'Bob'},
                        '13:00-17:00': {'Office Lead': None,
                                        'Officers': ['Alice'],
                                        'Shift Lead': 'Bob'},
                     

In [6]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.6 MB/s eta 0:00:00


In [7]:
import streamlit as st
from datetime import datetime
import random
from collections import defaultdict

# --- Backend Logic ---

WEEKDAYS = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
SHIFTS = [("09:00", "13:00"), ("13:00", "17:00"), ("17:00", "21:00")]

# Global state to hold submitted students
if "students" not in st.session_state:
    st.session_state.students = []

def time_overlap(start1, end1, start2, end2):
    return not (end1 <= start2 or end2 <= start1)

def is_available(class_times, shift_start, shift_end):
    shift_start = datetime.strptime(shift_start, "%H:%M").time()
    shift_end = datetime.strptime(shift_end, "%H:%M").time()
    for cls_start, cls_end in class_times:
        cls_start = datetime.strptime(cls_start, "%H:%M").time()
        cls_end = datetime.strptime(cls_end, "%H:%M").time()
        if time_overlap(shift_start, shift_end, cls_start, cls_end):
            return False
    return True

def assign_shifts(students):
    schedule = defaultdict(lambda: defaultdict(dict))
    shift_counts = defaultdict(int)

    for day in WEEKDAYS:
        for shift_start, shift_end in SHIFTS:
            available = [
                s for s in students
                if is_available(s["class_schedule"].get(day, []), shift_start, shift_end)
                and shift_counts[s["name"]] < 5
            ]
            random.shuffle(available)
            roles = {"Shift Lead": None, "Office Lead": None, "Officers": []}
            for student in available:
                if shift_counts[student["name"]] >= 5:
                    continue
                if student["role"] == "Shift Lead" and not roles["Shift Lead"]:
                    roles["Shift Lead"] = student["name"]
                    shift_counts[student["name"]] += 1
                elif student["role"] == "Office Lead" and not roles["Office Lead"]:
                    roles["Office Lead"] = student["name"]
                    shift_counts[student["name"]] += 1
                elif student["role"] == "Enforcement Officer" and len(roles["Officers"]) < 3:
                    roles["Officers"].append(student["name"])
                    shift_counts[student["name"]] += 1
            schedule[day][f"{shift_start}-{shift_end}"] = roles
    return schedule

# --- Streamlit UI ---

st.title("🚗 Parking Office Staff Scheduler")

st.header("👤 Add New Staff Member")
with st.form(key="student_form"):
    name = st.text_input("Name")
    role = st.selectbox("Role", ["Enforcement Officer", "Shift Lead", "Office Lead"])

    class_schedule = {}
    for day in WEEKDAYS:
        intervals = st.text_input(f"{day} Classes (e.g. 10:00-11:30,14:00-15:30)", key=day)
        slots = []
        if intervals:
            for part in intervals.split(","):
                times = part.strip().split("-")
                if len(times) == 2:
                    slots.append((times[0], times[1]))
        class_schedule[day] = slots

    submitted = st.form_submit_button("Add Student")
    if submitted and name:
        st.session_state.students.append({
            "name": name,
            "role": role,
            "class_schedule": class_schedule
        })
        st.success(f"{name} added!")

# --- View List ---
st.subheader("🧾 Current Staff Entries")
for s in st.session_state.students:
    st.markdown(f"**{s['name']}** - {s['role']}")

# --- Scheduler ---
if st.button("🧠 Generate Weekly Schedule"):
    if not st.session_state.students:
        st.warning("Please add at least one staff member.")
    else:
        schedule = assign_shifts(st.session_state.students)
        for day in WEEKDAYS:
            st.markdown(f"### 📅 {day}")
            for shift, roles in schedule[day].items():
                st.markdown(f"**{shift}**")
                st.markdown(f"- Shift Lead: {roles['Shift Lead'] or 'N/A'}")
                st.markdown(f"- Office Lead: {roles['Office Lead'] or 'N/A'}")
                st.markdown(f"- Enforcement Officers: {', '.join(roles['Officers']) if roles['Officers'] else 'N/A'}")


2025-04-20 23:44:31.591 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-20 23:44:31.598 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2025-04-20 23:44:31.602 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-20 23:44:31.606 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-20 23:44:31.612 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-20 23:44:31.756 
  command:

    streamlit run /usr/local/lib/python3.11/

In [10]:
!streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.138.247.40:8501

  Stopping...
  Stopping...
